# Продвинутое машинное обучение: Домашнее задание 3

### 1. Реализуйте базовый частотный метод по Шерлоку Холмсу:
 - подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
 - возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
 - расшифруйте их таким частотным методом.


In [1]:
from collections import Counter, defaultdict, OrderedDict
import re 
from scipy.special import softmax
import numpy as np
import tqdm
import random

In [2]:
f = open('WarAndPeace.txt', encoding="UTF-8")

In [3]:
full_text = f.read()

In [4]:
f.close()

In [5]:
# оставляем только буквы русского алфавита и пробелы
full_text_clean = re.findall('[а-я ]', full_text.lower().strip())

In [6]:
full_text_clean = ''.join(full_text_clean)

In [7]:
#удаляем лишние пробелы
full_text_clean = re.sub(' +', ' ', full_text_clean)

In [8]:
full_text_clean[:100]

'война и мир самый известный роман льва николаевича толстого как никакое другое произведение писателя'

**наш "контрольный" текст**

In [9]:
base_text = 'Быть энтузиасткой сделалось ее общественным положением, и иногда, когда ей даже того не хотелось, она, чтобы не обмануть ожиданий людей, знавших ее, делалась энтузиасткой. Сдержанная улыбка, игравшая постоянно на лице Анны Павловны, хотя и не шла к ее отжившим чертам, выражала, как у избалованных детей, постоянное сознание своего милого недостатка, от которого она не хочет, не может и не находит нужным исправляться.'
print(base_text)

Быть энтузиасткой сделалось ее общественным положением, и иногда, когда ей даже того не хотелось, она, чтобы не обмануть ожиданий людей, знавших ее, делалась энтузиасткой. Сдержанная улыбка, игравшая постоянно на лице Анны Павловны, хотя и не шла к ее отжившим чертам, выражала, как у избалованных детей, постоянное сознание своего милого недостатка, от которого она не хочет, не может и не находит нужным исправляться.


In [10]:
base_text_clean = re.findall('[а-я ]', base_text.lower().replace('  ', ' ').strip())

base_text_clean = ''.join(base_text_clean)

base_text_clean = re.sub(' +', ' ', base_text_clean)

print(base_text_clean)

быть энтузиасткой сделалось ее общественным положением и иногда когда ей даже того не хотелось она чтобы не обмануть ожиданий людей знавших ее делалась энтузиасткой сдержанная улыбка игравшая постоянно на лице анны павловны хотя и не шла к ее отжившим чертам выражала как у избалованных детей постоянное сознание своего милого недостатка от которого она не хочет не может и не находит нужным исправляться


In [11]:
'''
класс шифрования посредством случайной перестановки символов
'''
class Permutations():
    def __init__(self, text = ''):
        self.text = text
        #self.mixed_dict = {}
        if len(self.text) > 0:
            pass
    
    def fit(self, text):
        assert len(text) > 0
        
        self.text = text
        dd = defaultdict()
        base_arr = sorted(list(set(self.text)))
        perm_arr = np.random.permutation(base_arr)
        for letter_1, letter_2 in zip(base_arr, perm_arr):
            dd[letter_1] = letter_2
        #print(dd)
        self.mixed_dict = dict(dd)
    
    
    def fit_transform(self, text):
        self.fit(text)
        perm_text = ''
        for letter in self.text:
            #perm_text.append(self.mixed_dict[letter])
            perm_text += self.mixed_dict[letter]
        return perm_text
    
    def get_mixed_dict(self):
        return self.mixed_dict
        

**проверим работу класса**

In [12]:
perm = Permutations()

In [13]:
perm.fit_transform('привет меня зовут Cаша'.lower())

'пирзмяуом нувтзшяуаеcе'

**а это наши перестановки**

In [14]:
perm.get_mixed_dict()

{' ': 'у',
 'c': 'а',
 'а': 'е',
 'в': 'з',
 'е': 'м',
 'з': 'в',
 'и': 'р',
 'м': 'о',
 'н': ' ',
 'о': 'т',
 'п': 'п',
 'р': 'и',
 'т': 'я',
 'у': 'ш',
 'ш': 'c',
 'я': 'н'}

**сделаем перестановку символов в нашем "контрольком" тексте**

In [15]:
base_text_perm = perm.fit_transform(base_text_clean)
base_text_perm

'шьоэкытохзрчиобщдкицмачащиэкммкщшемиоумттьскйщащжмтрмскркртщгцчкбщгцчкмдкцчжмкощгщктмкющомащиэкщтчкпощшьктмкщшсчтхоэкщжрцчтрдкаяцмдкзтчулрюкммкцмачачиэкытохзрчиобщдкицм жчттчнкхаьшбчкрг чулчнкйщиощнттщктчкарвмкчттькйчуащутькющонкрктмклачкбкммкщожрулрскпм очскуь чжчачкбчбкхкрзшчащучттьюкцмомдкйщиощнттщмкищзтчтрмкиущмгщксращгщктмцщиочобчкщокбщощ щгщкщтчктмкющпмоктмксщжмокрктмктчющцроктхжтьскрий чуаноэин'

**подсчитаем частоты букв**

In [16]:
'''
функция подсчета частоты букв 
'''
def get_freq_single(txt):
    txt_len = len(txt)
    cnt = Counter(txt)
    out_freq = defaultdict(float)
    for i in cnt:
        out_freq[i] = cnt[i] / txt_len
    return out_freq

In [17]:
#частоты для всего текста
full_text_freq = get_freq_single(full_text_clean)
sorted(full_text_freq.items())

[(' ', 0.15933646611554253),
 ('а', 0.07055626912794517),
 ('б', 0.014529825158290818),
 ('в', 0.038742038639034505),
 ('г', 0.01744359353321337),
 ('д', 0.025574677214705868),
 ('е', 0.06635807045170433),
 ('ж', 0.008521250844711909),
 ('з', 0.014985540404931089),
 ('и', 0.05593124318182882),
 ('й', 0.009691752334370137),
 ('к', 0.030164603722819004),
 ('л', 0.042570358844543355),
 ('м', 0.024877058326869563),
 ('н', 0.0548091224204098),
 ('о', 0.09564089638564746),
 ('п', 0.021610578836396666),
 ('р', 0.03834562880120359),
 ('с', 0.043898487868142225),
 ('т', 0.047786113482460424),
 ('у', 0.02411857336157103),
 ('ф', 0.0018868484013290655),
 ('х', 0.007179075803237139),
 ('ц', 0.003400696994620375),
 ('ч', 0.011469353929997768),
 ('ш', 0.007943803443147181),
 ('щ', 0.002362852340456745),
 ('ъ', 0.0004416692287643718),
 ('ы', 0.01597032232489688),
 ('ь', 0.016383899517909452),
 ('э', 0.002542329235537674),
 ('ю', 0.005454536941807348),
 ('я', 0.0194724627819543)]

In [18]:
#частоты букв в контрольком тексте
base_text_freq = get_freq_single(base_text_perm)
sorted(base_text_freq.items())

[(' ', 0.01485148514851485),
 ('а', 0.03712871287128713),
 ('б', 0.022277227722772276),
 ('в', 0.0024752475247524753),
 ('г', 0.017326732673267328),
 ('д', 0.01485148514851485),
 ('е', 0.0024752475247524753),
 ('ж', 0.019801980198019802),
 ('з', 0.012376237623762377),
 ('и', 0.03712871287128713),
 ('й', 0.012376237623762377),
 ('к', 0.15099009900990099),
 ('л', 0.009900990099009901),
 ('м', 0.08168316831683169),
 ('н', 0.017326732673267328),
 ('о', 0.0594059405940594),
 ('п', 0.007425742574257425),
 ('р', 0.04702970297029703),
 ('с', 0.019801980198019802),
 ('т', 0.08663366336633663),
 ('у', 0.024752475247524754),
 ('х', 0.01485148514851485),
 ('ц', 0.027227722772277228),
 ('ч', 0.08415841584158416),
 ('ш', 0.01485148514851485),
 ('щ', 0.10148514851485149),
 ('ы', 0.0049504950495049506),
 ('ь', 0.022277227722772276),
 ('э', 0.01485148514851485),
 ('ю', 0.01485148514851485),
 ('я', 0.0024752475247524753)]

In [19]:
'''
функция сопоставления наиболее "похожих" частот
'''
def get_most_similar(letter, letter_dict, base_dict):
    
    delta = 1
    idx = -1
    
    letter_freq = letter_dict[letter]
    
    for i in base_dict:
        if abs(base_dict[i] - letter_freq) < delta:
            delta = abs(base_dict[i] - letter_freq)
            idx = i    
    
    return idx #, base_dict[idx], letter_freq

In [20]:
encoded_text = ''
for base_text_letter in base_text_perm:
    encoded_text += get_most_similar(base_text_letter, base_text_freq, full_text_freq)
print(encoded_text)

зпиз юоизчторипоз рдаророрз аа озэаримаоопя чорояаотая т тоогдо погдо аз дояа иого оа зоиарорз ооо хиозп оа озяоозиз оятдоотз рэдаз чоомйтз аа даророрз юоизчторипоз рдазяоооог зрпзпо тгзомйог чориогооо оо ртэа оооп чомромоп зоиг т оа йро п аа оиятмйтя хазиоя мпзояоро поп з тчзоромооопз даиаз чориогоооа рочооота рмоаго ятрого оадориоипо ои поиозого ооо оа зохаи оа яояаи т оа оозодти озяопя трчзомргизрг


In [21]:
'''
функция подсчета точности расшифрованного текста
'''
def calc_text_accuracy(base_text, predicted_text):
    cnt = 0
    for letter_1, letter_2 in zip(base_text, predicted_text):
        if letter_1 == letter_2:
            cnt += 1
    return cnt / len(base_text)

In [22]:
calc_text_accuracy(base_text_clean, encoded_text)

0.297029702970297

**Точность получилась так себе. Я решил попробовать улучшить этот результат - мне показалось отличной идеей нормализовать полученные частоты к распределению вероятностей**

In [23]:
'''
softmax
'''
def get_prob_single(dict_of_freq):
    dd = defaultdict()
    probs = softmax([i[1] for i in dict_of_freq.items()])
    for num, i in enumerate(dict_of_freq.items()):
        dd[i[0]] = probs[num]
        #print(num)
    return dd

In [24]:
full_text_prob = get_prob_single(full_text_freq)

In [25]:
base_text_prob = get_prob_single(base_text_freq)

In [26]:
base_text_prob

defaultdict(None,
            {'ш': 0.03168254901927859,
             'ь': 0.03191869115263402,
             'о': 0.03312606652012517,
             'э': 0.03168254901927859,
             'к': 0.036303154526003766,
             'ы': 0.031370408216541815,
             'т': 0.034040405067828766,
             'х': 0.03168254901927859,
             'з': 0.03160422384532779,
             'р': 0.03271861699617862,
             'ч': 0.033956250833689625,
             'и': 0.032396268710142354,
             'б': 0.03191869115263402,
             'щ': 0.034549728380340164,
             'д': 0.03168254901927859,
             'ц': 0.032077096243472866,
             'м': 0.0338723046445226,
             'а': 0.032396268710142354,
             'е': 0.031292854712908845,
             'у': 0.03199779567515125,
             'с': 0.03183978219125941,
             'й': 0.03160422384532779,
             'ж': 0.03183978219125941,
             'г': 0.031761068307563735,
             'ю': 0.03168254901927859

In [27]:
encoded_text = ''
for base_text_letter in base_text_perm:
    encoded_text += get_most_similar(base_text_letter, base_text_prob, full_text_prob)
print(encoded_text)

ааоа е оаао ооа а оо о о оа     ае ооо   аа а о а  о а о о  ао  а ао   а о а  о а     а о о оа     ео аа     аа  аоа  аоо  оа оео а а  оаоа    о о о оа е оаао ооа а оо аа    а аоааа  оаа оа а а оо а       оое     а а оо о а а оа о    ао  а     оаооаоа е ао а оаа а о  а а а оаа о о   аа о о а а оо а     о а   о  оо  а  аоо а    о оо оа   о а о а а         а е о    а а о о      а ооо  аа аа ооаа ооаоаоа


In [28]:
calc_text_accuracy(base_text_clean, encoded_text)

0.15099009900990099

**Получилось совсем плохо. Тогда я решил сделать ранжирование букв - ведь по сути частоты могут отличаться, но, например, пробелов будет больше всего, затем по частоте будет буква "о"...**

In [29]:
'''
функция ранжирования букв по частоте
'''
def get_numerate_dict(dict_of_freq):
    dd = defaultdict()
    ordered_dict = dict(sorted(dict_of_freq.items(), key = lambda kv:kv[1], reverse = True))
    for num, i in enumerate(ordered_dict.items()):
        dd[i[0]] = num
        #print(num)
    return dd

In [30]:
base_text_num = get_numerate_dict(base_text_freq)
print(base_text_num)

defaultdict(None, {'к': 0, 'щ': 1, 'т': 2, 'ч': 3, 'м': 4, 'о': 5, 'р': 6, 'и': 7, 'а': 8, 'ц': 9, 'у': 10, 'ь': 11, 'б': 12, 'с': 13, 'ж': 14, 'г': 15, 'н': 16, 'ш': 17, 'э': 18, 'х': 19, 'д': 20, 'ю': 21, ' ': 22, 'з': 23, 'й': 24, 'л': 25, 'п': 26, 'ы': 27, 'е': 28, 'я': 29, 'в': 30})


In [31]:
full_text_num = get_numerate_dict(full_text_freq)
print(full_text_num)

defaultdict(None, {' ': 0, 'о': 1, 'а': 2, 'е': 3, 'и': 4, 'н': 5, 'т': 6, 'с': 7, 'л': 8, 'в': 9, 'р': 10, 'к': 11, 'д': 12, 'м': 13, 'у': 14, 'п': 15, 'я': 16, 'г': 17, 'ь': 18, 'ы': 19, 'з': 20, 'б': 21, 'ч': 22, 'й': 23, 'ж': 24, 'ш': 25, 'х': 26, 'ю': 27, 'ц': 28, 'э': 29, 'щ': 30, 'ф': 31, 'ъ': 32})


In [32]:
#perm.get_mixed_dict()

In [33]:
encoded_text = ''
for base_text_letter in base_text_perm:
    encoded_text += get_most_similar(base_text_letter, base_text_num, full_text_num)
print(encoded_text)

гкнь юаныйтесндоз свилелось ии огциснриаакм жолоуиатим т таопве допве из веуи нопо аи бонилось оае хногк аи огмеаынь оутвеатз лэвиз йаерштб ии вилелесь юаныйтесндоз свичуеааея ылкгде тпчершея жоснояаао ае лтщи еаак жерлорак боня т аи шле д ии онутрштм хичнем ркчеуеле дед ы тйгелореаакб виниз жоснояааои сойаеати сроипо мтлопо аивосненде он доночопо оае аи бохин аи моуин т аи аебовтн аыуакм тсжчерлянься


In [34]:
calc_text_accuracy(base_text_clean, encoded_text)

0.3886138613861386

**0,388! прошлый 0.297 - результат улучшился! Воодушевленный успехом я решил, что вот оно! Сейчас все получится!**

---

### 2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

 - подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
 - проведите тестирование аналогично п.1, но при помощи биграмм.


In [35]:
'''
функция подсчета частоты биграмм 
'''
def get_freq_bigramm(txt):
    txt_len = len(txt) - 1
    out_freq = defaultdict(float)
    for i in range(txt_len):
        out_freq[txt[i:i+2]] += 1
    for i in out_freq:
        out_freq[i] = out_freq[i] / txt_len
    return out_freq

In [36]:
#частоты для всего текста
full_text_bi_freq = get_freq_bigramm(full_text_clean)
sorted(full_text_bi_freq.items())[:10]

[(' а', 0.0028841201716738197),
 (' б', 0.007249317206398751),
 (' в', 0.014765509168942646),
 (' г', 0.005042528287163481),
 (' д', 0.006781115879828326),
 (' е', 0.0038017947717518532),
 (' ж', 0.0016574326960593056),
 (' з', 0.004054623488099883),
 (' и', 0.01051736246586032),
 (' й', 2.6531408505657433e-05)]

In [37]:
#частоты букв в контрольком тексте
base_text_bi_freq = get_freq_bigramm(base_text_perm)
sorted(base_text_bi_freq.items())[:10]

[(' ж', 0.0024813895781637717),
 (' о', 0.0024813895781637717),
 (' ч', 0.007444168734491315),
 (' щ', 0.0024813895781637717),
 ('ан', 0.0024813895781637717),
 ('ар', 0.0024813895781637717),
 ('ач', 0.01240694789081886),
 ('ащ', 0.01488833746898263),
 ('аь', 0.0024813895781637717),
 ('ая', 0.0024813895781637717)]

In [38]:
encoded_text = ''
for base_text_bigramm in base_text_perm:
    encoded_text += get_most_similar(base_text_bigramm, base_text_bi_freq, full_text_bi_freq)
print(encoded_text)

иэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэиэ


**Получилось очень плохо. Все из-за маленького объема текста - просто очень мало биграмм и их частоты с реальными вообще не совпадают. Я пробовал брать большие тексты - на уровне книги - результат вполне сносный, но в нашем случае такой вариант не подойдет, потому что наша цель расшифровать короткий текст.**

---

### 3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
 - предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
 - реализуйте и протестируйте его, убедитесь, что результаты улучшились.


Пусть текст - это цепь Маркова, где буквы являются состояниями. Нужно обучить матрицу переходов между буквами на тренировочном тексте. Далее для каждой перестановки, с помощью которой можно осуществить расшифровку, можно найти правдоподобие этой перестановки, перемножив вероятности всех переходов в расшифрованном тексте. Соответственно можно применить MCMC-семплирование для выбора перестановки из распределения, заданного этим правдоподобием.

In [39]:
class MCMCDecoder:
    def __init__(self):
        self.alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
        self.char_to_idx = {char: idx for idx, char in enumerate(self.alphabet)}
        self.transition_matrix = np.zeros((len(self.alphabet), len(self.alphabet)))
    
    def fit(self, text):
        text = ''.join([char for char in text.lower() if char in self.alphabet])
        for i in tqdm.tqdm(range(len(text) - 1), position=0, leave=True):
            self.transition_matrix[self.char_to_idx[text[i]], self.char_to_idx[text[i+1]]] += 1
        self.transition_matrix = np.clip(self.transition_matrix, 1, None)
        self.transition_matrix = (np.log(self.transition_matrix).T 
                                   - np.log(self.transition_matrix.sum(axis=1))).T
        
    def translate(self, text, permutation):
        transtable = str.maketrans(self.alphabet, ''.join(permutation))
        return text.translate(transtable)
        
    def evaluate_log_likelihood(self, text, permutation):
        text = self.translate(text, permutation)
        log_likelihood = 0
        for i in range(len(text) - 1):
            log_likelihood += self.transition_matrix[self.char_to_idx[text[i]], self.char_to_idx[text[i+1]]]
        return log_likelihood
    
    def transform(self, text, n_iters=10000):
        permutation = np.array(list(self.alphabet))
        random.shuffle(permutation)
        current_log_likelihood = self.evaluate_log_likelihood(text, permutation)
        best_log_likelihood = current_log_likelihood
        best_permutation = permutation.copy()
        third_part = n_iters // 3
        for i in tqdm.tqdm(range(n_iters), position=0, leave=True):
            if i % third_part == 0:
                random.shuffle(permutation)
                current_log_likelihood = self.evaluate_log_likelihood(text, permutation)
            swap_idx = random.sample(range(len(self.alphabet)), 2)
            permutation[swap_idx[0]], permutation[swap_idx[1]] = \
                permutation[swap_idx[1]], permutation[swap_idx[0]]
            new_log_likelihood = self.evaluate_log_likelihood(text, permutation)
            if new_log_likelihood >= current_log_likelihood:
                current_log_likelihood = new_log_likelihood
                if new_log_likelihood > best_log_likelihood:
                    best_log_likelihood = new_log_likelihood
                    best_permutation = permutation.copy()
            else:
                if random.random() < np.exp(new_log_likelihood - current_log_likelihood):
                    current_log_likelihood = new_log_likelihood
                else:
                    permutation[swap_idx[0]], permutation[swap_idx[1]] = \
                        permutation[swap_idx[1]], permutation[swap_idx[0]]
        return self.translate(text, best_permutation)

In [40]:
mcmc = MCMCDecoder()

In [41]:
# обучим наш декодер на всем тексте
mcmc.fit(full_text_clean)

100%|██████████████████████████████████████████████████████████████████████| 640750/640750 [00:02<00:00, 242765.90it/s]


In [52]:
#и сделаем тестовую расшифровку контрольного текста 
encoded_text_mcmc = mcmc.transform(base_text_perm)

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:07<00:00, 1284.43it/s]


In [53]:
encoded_text_mcmc

'быть чнтужиастрой смелалось ее объественных полодениех и инозма розма ей маде тозо не котелось она фтобы не обхануть одиманий люмей жнавшик ее мелалась чнтужиастрой смегданная улыбра изгавшая постоянно на лице анны павловны котя и не шла р ее отдивших фегтах выгадала рар у ижбалованнык метей постоянное сожнание своезо хилозо немостатра от ротогозо она не кофет не ходет и не накомит нудных испгавляться'

In [54]:
calc_text_accuracy(base_text_clean, encoded_text_mcmc)

0.8366336633663366

**результат улучшается! Возможно нужно больше итераций**

In [55]:
acc_max = 0
i_max = -1
best_text = ''
for i in tqdm.tqdm(range(3000, 30000, 3000), position=0, leave=True):
    encoded_text_mcmc = mcmc.transform(base_text_perm, n_iters=i)
    acc = calc_text_accuracy(base_text_clean, encoded_text_mcmc)
    if acc > acc_max:
        i_max = i
        acc_max = acc
        best_text = encoded_text_mcmc
    if acc_max > 0.9:
        break        

 44%|█████████████████████████████████████▎                                              | 4/9 [00:34<00:43,  8.73s/it]


In [56]:
best_text

'быть энтузиасткой сделалось ее объественным положением и иногда когда ей даже того не хотелось она чтобы не обмануть ожиданий людей знавших ее делалась энтузиасткой сдержанная улыбка игравшая постоянно на лице анны павловны хотя и не шла к ее отжившим чертам выражала как у избалованных детей постоянное сознание своего милого недостатка от которого она не хочет не может и не находит нужным исправляться'

In [57]:
acc_max

0.9975247524752475

**Отличный результат. При чтении мне пришла идея, что метрику качества можно считать как долю расшифрованных слов, присутствующих в базовом корпусе, на котором мы считали частоты - тогда мы сможем это использовать и при расшифровывании главного сообщения**

In [58]:
'''
функция подсчета "действительных" (валидных) слов 
'''
def calc_valid_word_accuracy(text, full_text):
    set_of_words = set(full_text.split(' '))
    cnt = 0
    words = text.split(' ')
    text_len = len(words)
    for word in words:
        if word in set_of_words:
            cnt += 1
    return cnt / text_len

In [59]:
acc_max = 0
i_max = -1
best_text = ''
for i in tqdm.tqdm(range(3000, 30000, 3000), position=0, leave=True):
    encoded_text_mcmc = mcmc.transform(base_text_perm, n_iters=i)
    acc = calc_valid_word_accuracy(encoded_text_mcmc, full_text_clean)
    if acc > acc_max:
        i_max = i
        acc_max = acc
        best_text = encoded_text_mcmc
    if acc_max > 0.8:
        break  

 44%|█████████████████████████████████████▎                                              | 4/9 [00:36<00:45,  9.08s/it]


In [60]:
acc_max

0.9354838709677419

In [61]:
encoded_text_mcmc

'быть щнтузиасткой сделалось ее объественным положением и иногда когда ей даже того не хотелось она чтобы не обмануть ожиданий людей знавших ее делалась щнтузиасткой сдержанная улыбка игравшая постоянно на лице анны павловны хотя и не шла к ее отжившим чертам выражала как у избалованных детей постоянное сознание своего милого недостатка от которого она не хочет не может и не находит нужным исправляться'

**получилось очень хорошо**

---

### 4. Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹
⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷
⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹
⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝
↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏

**попробуем теперь что получится на "боевом" тексте**

In [62]:
homework_text = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

**Переведём сначала в алфавит, аналогичный тому на котором расшифровывается сообщение**

In [63]:
homework_alphabet = ''.join(set(homework_text))
homework_alphabet

'↨⇊↞↲↝↘↙⇆⇙⇞↹⇛⇏←⇠↟⇯⇴↾↷↳⇷↤⇒⇈⇸⇰⇌'

In [64]:
rus_alphabet_short = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '[:len(homework_alphabet)]

In [65]:
transtable = str.maketrans(homework_alphabet, rus_alphabet_short)

In [66]:
homework_text_rus = homework_text.translate(transtable)
homework_text_rus

'мнцойтбйтохоамйдкпурцчдбсйоцойфкиаойдкпурцчдбсйамънайзйшакёкйнккгжмдовйъкакпбсйцмёъкйфпкиоарачйнъкпммйтнмёкйтбйтнмйнхмцрцойфпртоцчдкйойфкцзиоамйуръноурцчдбсйгрццйщрйфкнцмхдммйиматмпакмйщрхрдомйъзпнрйекавйъкдмидкйвйдоимёкйдмйкгмжрл'

In [67]:
#message_decoded, _ = mcmc.transform(homework_text_rus, n_iters=30000)

In [68]:
acc_max = 0
i_max = -1
best_text = ''
for i in tqdm.tqdm(range(10000, 30000, 1000), position=0, leave=False):
    encoded_text_mcmc = mcmc.transform(homework_text_rus, n_iters=i)
    acc = calc_valid_word_accuracy(encoded_text_mcmc, full_text_clean)
    if acc > acc_max:
        i_max = i
        acc_max = acc
        best_text = encoded_text_mcmc
    if acc_max > 0.8:
        break   

100%|██████████████████████████████████████████████████████████████████████████| 29000/29000 [00:13<00:00, 2126.94it/s]


In [69]:
acc_max

0.6

In [70]:
best_text

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаш'

**Текст вполне читаем, это радует. Для улучшения точности попробуем на другом корпусе. Я взял "Преступление и наказание" Достоевского.**

In [74]:
f = open('Dostoevskiy_Fedor_Prestuplenie_i_nakazanie.txt', encoding="windows-1251")
full_text = f.read()
f.close()

# оставляем только буквы русского алфавита и пробелы
full_text_clean = re.findall('[а-я ]', full_text.lower())

full_text_clean = ''.join(full_text_clean)

#удаляем лишние пробелы
full_text_clean = re.sub(' +', ' ', full_text_clean)

full_text_clean[:100]

'преступление и наказание достоевскогороман фмдостоевского преступление и наказание принадлежит к чис'

In [75]:
mcmc.fit(full_text_clean)

100%|████████████████████████████████████████████████████████████████████| 1114021/1114021 [00:03<00:00, 316938.47it/s]


In [76]:
acc_max = 0
i_max = -1
best_text = ''
for i in tqdm.tqdm(range(10000, 30000, 1000), position=0, leave=False):
    encoded_text_mcmc = mcmc.transform(homework_text_rus, n_iters=i)
    acc = calc_valid_word_accuracy(encoded_text_mcmc, full_text_clean)
    if acc > acc_max:
        i_max = i
        acc_max = acc
        best_text = encoded_text_mcmc
    if acc_max > 0.8:
        break   

100%|██████████████████████████████████████████████████████████████████████████| 29000/29000 [00:13<00:00, 2132.57it/s]


In [77]:
acc_max

0.7142857142857143

In [78]:
best_text

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

**Да, результат улучшился! Ошибок в тексте очень мало и текст можно прочитать без проблем**